In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.20.0


# Load the dataset

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step 
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Normalize pixel values to [0, 1]

In [3]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# CNN requires an explicit channel dimension (N, H, W, C)

In [4]:
x_train_cnn = np.expand_dims(x_train, -1)
x_test_cnn = np.expand_dims(x_test, -1)

print(f"Original training data shape: {x_train.shape}")
print(f"Shape for MLP (will be flattened by layer): {x_train.shape}")
print(f"Shape for CNN (with channels): {x_train_cnn.shape}")

Original training data shape: (60000, 28, 28)
Shape for MLP (will be flattened by layer): (60000, 28, 28)
Shape for CNN (with channels): (60000, 28, 28, 1)


# Implement and Compile the MLP Model

In [6]:
mlp_model = keras.Sequential(
    [
        keras.Input(shape=(28, 28)), 
        
        layers.Flatten(), 
        
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ],
    name="mlp_model",
)

# Compile the MLP model

In [7]:

mlp_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)


# Print MLP summary

In [8]:
mlp_model.summary()


Model: "mlp_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │         200,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

# Implement and Compile the CNN Model

In [11]:
cnn_model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)), 
        
        # Conv Block 1 
        layers.Conv2D(16, kernel_size=3, activation="relu"),
        layers.MaxPooling2D(pool_size=2),
        
        # Conv Block 2
        layers.Conv2D(32, kernel_size=3, activation="relu"),
        layers.MaxPooling2D(pool_size=2),
        
        # Classifier
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ],
    name="cnn_model",
)


# Compile the CNN model

In [12]:

cnn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Print CNN summary


In [13]:
cnn_model.summary()


Model: "cnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 16)          │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)

# Training and Evaluation

In [15]:
EPOCHS = 25
BATCH_SIZE = 32

# Training MLP Model

In [16]:
mlp_history = mlp_model.fit(
    x_train, 
    y_train, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_split=0.1 # Using 10% for validation
)

Epoch 1/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8289 - loss: 0.4811 - val_accuracy: 0.8595 - val_loss: 0.3890
Epoch 2/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8651 - loss: 0.3637 - val_accuracy: 0.8680 - val_loss: 0.3719
Epoch 3/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8784 - loss: 0.3256 - val_accuracy: 0.8752 - val_loss: 0.3411
Epoch 4/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8869 - loss: 0.3017 - val_accuracy: 0.8778 - val_loss: 0.3366
Epoch 5/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8943 - loss: 0.2827 - val_accuracy: 0.8728 - val_loss: 0.3398
Epoch 6/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8994 - loss: 0.2688 - val_accuracy: 0.8842 - val_loss: 0.3173
Epoch 7/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9053 - loss: 0.2530 - val_accuracy: 0.8793 - val_loss: 0.3496
Epoch 8/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9088 - loss: 0.2430 - 

# Training CNN Model

In [17]:
cnn_history = cnn_model.fit(
    x_train_cnn, 
    y_train, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_split=0.1
)

Epoch 1/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8100 - loss: 0.5295 - val_accuracy: 0.8542 - val_loss: 0.3915
Epoch 2/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8702 - loss: 0.3567 - val_accuracy: 0.8722 - val_loss: 0.3469
Epoch 3/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8859 - loss: 0.3134 - val_accuracy: 0.8882 - val_loss: 0.3101
Epoch 4/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8970 - loss: 0.2839 - val_accuracy: 0.8948 - val_loss: 0.2831
Epoch 5/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9052 - loss: 0.2619 - val_accuracy: 0.9013 - val_loss: 0.2786
Epoch 6/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9106 - loss: 0.2417 - val_accuracy: 0.9003 - val_loss: 0.2774
Epoch 7/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9170 - loss: 0.2246 - val_accuracy: 0.9063 - val_loss: 0.2614
Epoch 8/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9222 - loss: 0

# Evaluate and Report

In [18]:
print("Evaluating MLP model on test data...")
mlp_loss, mlp_acc = mlp_model.evaluate(x_test, y_test)
print(f"MLP Test Accuracy: {mlp_acc:.4f}, Test Loss: {mlp_loss:.4f}")

print("\nEvaluating CNN model on test data...")
cnn_loss, cnn_acc = cnn_model.evaluate(x_test_cnn, y_test)
print(f"CNN Test Accuracy: {cnn_acc:.4f}, Test Loss: {cnn_loss:.4f}")

Evaluating MLP model on test data...
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8915 - loss: 0.4359
MLP Test Accuracy: 0.8915, Test Loss: 0.4359

Evaluating CNN model on test data...
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8984 - loss: 0.4114
CNN Test Accuracy: 0.8984, Test Loss: 0.4114


# Save models in Keras .h5 format

In [22]:
mlp_model.save("mlp_model.h5")
cnn_model.save("cnn_model.h5")

# Get file size

In [23]:
mlp_size = os.path.getsize("mlp_model.h5") / (1024 * 1024) # in MB
cnn_size = os.path.getsize("cnn_model.h5") / (1024 * 1024) # in MB

print(f"MLP Model Size (mlp_model.h5): {mlp_size:.2f} MB")
print(f"CNN Model Size (cnn_model.h5): {cnn_size:.2f} MB")

MLP Model Size (mlp_model.h5): 2.72 MB
CNN Model Size (cnn_model.h5): 0.69 MB
